In [1]:
# %%capture
# import torch
# major_version, minor_version = torch.cuda.get_device_capability()
# # Must install separately since Colab has torch 2.2.1, which breaks packages
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# if major_version >= 8:
#     # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
#     !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
# else:
#     # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
#     !pip install --no-deps xformers trl peft accelerate bitsandbytes
# pass

In [26]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 82.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


### Load Model

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 20 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'unsloth/llama-3-8b-Instruct-bnb-4bit',
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA A30 MIG 2g.12gb. Max memory: 11.688 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Add LoRA adapters to update 1 to 10% of all parameters

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Preparation

In [3]:
from datasets import load_dataset

dataset = load_dataset(
    "msc-smart-contract-audition/audits-with-reasons",
)

print(dataset)

Generating train split:   0%|          | 0/2472 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/437 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['code', 'description', 'recommendation', 'type', 'functionality'],
        num_rows: 2472
    })
    test: Dataset({
        features: ['code', 'description', 'recommendation', 'type', 'functionality'],
        num_rows: 437
    })
})


In [4]:
def unescape_newlines(cell):
    if cell is None:
        return None
    return cell.replace("\\n", "\n")

train_dataset = dataset["train"].to_pandas().map(unescape_newlines)
test_dataset = dataset["test"].to_pandas().map(unescape_newlines)


In [18]:
print(train_dataset.iloc[0])

code              ```\nrequire(block.timestamp >= _d.withdrawalR...
description       The `increaseRedemptionFee` function in the `D...
recommendation    To mitigate the vulnerability, it is essential...
type                                                     reentrancy
functionality     Code block 1:\n1.  This code checks if the cur...
Name: 0, dtype: object


In [8]:
print(train_dataset.iloc[30]['description'])
print('====')
print(train_dataset.iloc[30]['recommendation'])

The vulnerability lies in the `market.update(account,max,max,max,0,false)` function, which is used to "settle" an account without changing its position or collateral. This action is assumed to be a reliable way to settle the account, but it can actually revert if the account is below the margin requirement. This can cause unexpected reverts and denial of service to users who cannot execute transactions in certain situations.

In the context of the Oracle Keeper, this issue arises when the `KeeperFactory.settle` function attempts to settle all accounts in the market for a specific oracle version. If any account being settled is below the margin requirement, the entire market version's settlement will revert, causing the keepers to temporarily be unable to call this function for the specific oracle version until all users are at or above the margin.

Similarly, in the context of the Vault, the `_updateUnderlying` function uses this method to settle all vault accounts in the markets. This

In [34]:
prompt_train = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below are one or more Solidity codeblocks. The codeblocks might contain vulnerable code.
If there is a vulnerability please provide a brief description of the vulnerability.
Otherwise, output "There is no vulnerability in the code".
Think step by step about the code and the possible vulnerabilities.
<|start_header_id|>user<|end_header_id|>
Codeblocks:
{}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
Vulnerability audit: {}<|eot_id|>
"""

prompt_test = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below are one or more Solidity codeblocks. The codeblocks might contain vulnerable code.
If there is a vulnerability please provide a brief description of the vulnerability.
Otherwise, output "There is no vulnerability in the code".
Think step by step about the code and the possible vulnerabilities.
<|start_header_id|>user<|end_header_id|>
Codeblocks:
{}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
Vulnerability audit:
"""

EOS_TOKEN = tokenizer.eos_token

def format_train(row):
    text = prompt_train.format(row["code"], row["description"])
    return { "text" : text, }

def format_test(data):
    return { "text" : prompt_test.format(data["code"]), }

formatted_train = train_dataset.apply(format_train, axis=1)
formatted_test = test_dataset.apply(format_test, axis=1)

IndexError: Replacement index 1 out of range for positional args tuple

In [33]:
print(formatted_train[67]['text'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below are one or more Solidity codeblocks. The codeblocks might contain vulnerable code.
If there is a vulnerability please provide a brief description of the vulnerability.
Otherwise, output "There is no vulnerability in the code".
Think step by step about the code and the possible vulnerabilities.
<|start_header_id|>user<|end_header_id|>
Codeblocks:
```
modifier refundsAttachedEth() {
    _;
    uint256 remainingBalance = LibSafeMathV06.min256(msg.value, address(this).balance);
    if (remainingBalance > 0) {
        msg.sender.transfer(remainingBalance);
    }
}
```
```
if (inputToken == ETH_TOKEN_ADDRESS) {
    provider.transfer(sellAmount);
```<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
Vulnerability audit: The vulnerability arises from a combination of conditions that allow an attacker to steal temporarily held ether during a transaction. The exchange proxy, which typically holds no ether balance, can tempo

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 20,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2): 100%|█████████████████████████████████| 2473/2473 [00:01<00:00, 2402.90 examples/s]
max_steps is given, it will override any value given in num_train_epochs


In [10]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A30 MIG 2g.12gb. Max memory = 11.688 GB.
4.016 GB of memory reserved.


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 20
 "-____-"     Number of trainable parameters = 39,976,960


Step,Training Loss
1,4.338700
2,4.338700
3,4.330300
4,4.245400
5,4.026900
6,3.574000
7,3.051000
8,2.447200
9,2.053200
10,1.588000


In [12]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

54.3298 seconds used for training.
0.91 minutes used for training.
Peak reserved memory = 4.381 GB.
Peak reserved memory for training = 0.365 GB.
Peak reserved memory % of max memory = 37.483 %.
Peak reserved memory for training % of max memory = 3.123 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [13]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
idx = 71
inputs = tokenizer([formatted_test[idx]['text']], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = max_seq_length, use_cache = True)

print(tokenizer.batch_decode(outputs))
print(formatted_test[idx]['severity'])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nThe solidity code might contain a vulnerabilitity. Respond with a single word that best describes the severity of the vulnerability.\nThe possible responses are:\n- "low"\n- "medium"\n- "high"\n- "none"\n\nDo not write anything else - no explanations or no reasons. Just output one of the 4 levels.\n\n### Function:\n```\nfunction converterType() public pure returns (uint16) {\n    return 3;\n}\n```\n\n\n### Response:\n```\nmedium\n```\n\n### Explanation:\nThe function returns a constant value']
medium


 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [15]:
# Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
# ['<s> Below is a solidity code which might contain a vulnerabilitity.\n\nRespond with a single word that best describes the severity of the vulnerability.\n\nThe possible responses are:\n- "low",\n- "medium"\n- "high"\n- "none", only if there is no vulnerability.\n\nRemember output only 1 word in your response.\n\n### Function:\n```\n/// @notice Starts signer liquidation due to abort or undercollateralization\n/// @dev We first attempt to liquidate on chain, then by auction\n/// @param \\_d deposit storage pointer\nfunction startSignerAbortLiquidation(DepositUtils.Deposit storage \\_d) internal {\n    \\_d.logStartedLiquidation(false);\n    // Reclaim used state for gas savings\n    \\_d.redemptionTeardown();\n    \\_d.seizeSignerBonds();\n\n    \\_d.liquidationInitiated = block.timestamp;  // Store the timestamp for auction\n    \\_d.liquidationInitiator = msg.sender;\n    \\_d.setFraudLiquidationInProgress();\n}\n```\n\n\n### Severity: The severity is .\n\n### Explanation:\n\nThe function `startSignerAbortLiquid']
# high


SyntaxError: invalid syntax (1725294777.py, line 1)

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

NameError: name 'alpaca_prompt' is not defined

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64)

["<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is a famous tall tower in Paris?\n\n### Input:\n\n\n### Response:\nThe Eiffel Tower is a famous tall tower in Paris. It is located on the Champ de Mars in the 7th arrondissement of Paris. It was built in 1889 as the main entrance to the 1889 World's Fair.</s>"]

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Mistral 7b 2x faster [free Colab](https://colab.research.google.com/drive/1Dyauq4kTZoLewQ1cApceUQVNcnnNTzg_?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. Gemma 6 trillion tokens is 2.5x faster! [free Colab](https://colab.research.google.com/drive/10NbwlsRChbma1v55m8LAPYG15uQv6HLo?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>